In [ ]:
USER = 'Paolo'

In [ ]:
import sys
import os
if (colab := 'google.colab' in sys.modules):
    from google.colab import drive
    drive.mount('/content/drive')
    # BASE_PATH = '/content/drive/Shareddrives/Project_ML_23/' + USER + '/machine-learning-project'
    BASE_PATH = '/content/drive/My Drive/machine-learning-project'
    sys.path.insert(0, BASE_PATH)
    N_JOBS = -1
    sys.path.insert(0, BASE_PATH + '/src/utils')
    !pip install optuna
    !pip install scikit-learn
    !pip install scikeras
    TRAIN_DATA = os.path.join(BASE_PATH, 'datasets', 'cup2023','ML-CUP23-TR.csv')
    IMAGES_FOLDER = os.path.join(BASE_PATH, 'images', 'cup2023', 'random_forest')
    MODEL_FOLDER = os.path.join(BASE_PATH, 'trained_models', 'cup2023')
else :
    N_JOBS = -1
    TRAIN_DATA = os.path.join('..', '..', 'datasets', 'cup2023', 'ML-CUP23-TR.csv')
    IMAGES_FOLDER = os.path.join('..', '..', 'images', 'cup2023', 'random_forest')
    MODEL_FOLDER = os.path.join('..', '..', 'trained_models', 'cup2023')

In [ ]:
if (colab := 'google.colab' in sys.modules):
    sys.path.append(BASE_PATH + '/src/utils')
else:
    sys.path.append('../utils')


import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style='darkgrid')

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import make_scorer
from utils import save_plot, mean_euclidean_error, root_mean_squared_error, multidim_r2, set_random_state, mean_squared_error
from joblib import dump

set_random_state(42)

In [ ]:
# To skip the first column (row indexes)
columns_to_read = list(range(1, 14))

df_train = pd.read_csv(TRAIN_DATA, header=None, comment='#', usecols=columns_to_read, delimiter=',')
df_train = df_train.astype('float64') #casting
df_train.head()

In [ ]:
features = ['feature_' + str(i) for i in range(1, 11)]
targets = ['target_x', 'target_y', 'target_z']

# Rename columns
new_column_names = features + targets
df_train.columns = new_column_names

df_train.head()

In [ ]:
from sklearn.model_selection import train_test_split

X = df_train[features].to_numpy()
y = df_train[targets].to_numpy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create model

In [ ]:
mee_scorer = make_scorer(mean_euclidean_error, greater_is_better=False)
rmse_scorer = make_scorer(root_mean_squared_error, greater_is_better=False)
multidim_r2_scorer = make_scorer(multidim_r2, greater_is_better=True)

scoring = {'MEE': mee_scorer, 'R2': multidim_r2_scorer, 'RMSE': rmse_scorer}

In [ ]:
model = RandomForestRegressor(random_state=42)

param_grid = {
    'rf__n_estimators': [100, 150, 200, 300],
    'rf__max_depth': [None, 4, 6, 8, 10],
    'rf__min_samples_split': [2, 8, 10],
    'rf__min_samples_leaf': [1, 3, 4],
    'rf__max_features': ['sqrt', 'log2'],
}

In [ ]:
# Definizione del pipeline con RobustScaler e un modello Ridge
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import RobustScaler

'''
Scales input data using RobustScaler with the formula: (X - median(X)) / IQR,
where IQR is the interquartile range (75th - 25th percentile). This scaler is
chosen for its robustness to outliers, utilizing the median rather than the mean.
It is particularly beneficial for algorithms that are sensitive to the scale of data,
like neural networks, where it promotes zero mean and unit variance, leading to faster
gradient descent convergence. It also improves algorithms based on distance measures
(e.g., K-nearest neighbors, hierarchical clustering) and those assuming data normality.
However, algorithms such as Random Forest, which do not rely on distance or data normality,
may not see as much benefit from scaling.

Parameters:
X_train (array-like): Input data to be scaled.

Returns:
scaled_data (array-like): Scaled version of X_train.
'''

pipeline = Pipeline([
    ('scaler', RobustScaler()),
    ('rf', model)
])

In [ ]:
from sklearn.model_selection import GridSearchCV, KFold

cv = KFold(n_splits=5, shuffle=True, random_state=42)

grid_search = GridSearchCV(
    pipeline,
    param_grid=param_grid,
    cv=cv,
    scoring=scoring,
    refit='MEE',
    n_jobs=-1,
    verbose=2
)
grid_search.fit(X_train, y_train)

In [ ]:
results = pd.DataFrame(grid_search.cv_results_)
results.to_csv(os.path.join(BASE_PATH, 'src', 'cup2023', 'grid_search_RF_results.csv'), index=False)

In [ ]:
'''
Quando si utilizza grid_search.best_score_ in scikit-learn e si ottiene un valore negativo,
ciò è tipicamente dovuto al fatto che scikit-learn trasforma alcune funzioni di scoring in modo
che possano essere trattate come funzioni di "massimizzazione". '''


best_params = grid_search.best_params_
best_index = grid_search.best_index_

mee = grid_search.best_score_
r2 = grid_search.cv_results_['mean_test_R2'][best_index]
rmse = grid_search.cv_results_['mean_test_RMSE'][best_index]

print('Best parameters:', best_params)
print('Best MEE score:', abs(mee))
print('R2:', abs(r2))
print('RMSE:', abs(rmse))

# Learning curve

In [ ]:
from sklearn.model_selection import learning_curve

best_params_cleaned = {key.replace('rf__', ''): value for key, value in best_params.items()}

estimator = Pipeline([
    ('scaler', RobustScaler()),
    ('rf', RandomForestRegressor(**best_params_cleaned))
])

train_sizes, train_scores, validation_scores = learning_curve(
    estimator=estimator,
    X=X_train,
    y=y_train,
    train_sizes=[0.1, 0.33, 0.55, 0.78, 1.],
    cv=5,
    n_jobs=N_JOBS,
    verbose=2,
    scoring=mee_scorer
)

In [ ]:
train_scores_mean = np.mean(np.abs(train_scores), axis=1)
train_scores_std = np.std(np.abs(train_scores), axis=1)
validation_scores_mean = np.mean(np.abs(validation_scores), axis=1)
validation_scores_std = np.std(np.abs(validation_scores), axis=1)

In [ ]:
print(train_scores_mean[-1])
print(train_scores_std[-1])
print(validation_scores_mean[-1])
print(validation_scores_std[-1])

In [ ]:
plt.figure()
plt.title('Random Forest Learning Curve', fontweight='bold', fontsize=16)
plt.xlabel('Training examples', fontweight='bold')
plt.ylabel('Absolute Error', fontweight='bold')
plt.grid(True)

color1 = sns.dark_palette((20, 60, 50), input='husl')[-1]
color2 = sns.dark_palette('seagreen')[-1]

# Filling the area around the mean scores to indicate variability of the model's performance
# The shaded area represents the range of scores (mean ± standard deviation) for each training set size
plt.fill_between(
    train_sizes, train_scores_mean - train_scores_std,
                 train_scores_mean + train_scores_std, alpha=0.2, color=color1
)
plt.fill_between(
    train_sizes, validation_scores_mean - validation_scores_std,
                 validation_scores_mean + validation_scores_std, alpha=0.2, color=color2
)

# Mean score lines for training and validation
sns.lineplot(x=train_sizes, y=train_scores_mean, marker='o', color=color1, label='Training score')
sns.lineplot(x=train_sizes, y=validation_scores_mean, marker='s', linestyle='-.', color=color2, label='Cross-validation score')

plt.legend(loc='upper right')

save_plot(plt, IMAGES_FOLDER, 'learning_curve')
plt.show()

# Test of the model

In [ ]:
i = 0
final_model = []

for train_index, test_index in cv.split(X_train):
    X_train_1, X_test_1 = X_train[train_index], X_train[test_index]
    y_train_1, y_test_1 = y_train[train_index], y_train[test_index]
    i = i + 1

    # Generate 5 possible splits and foreach save a model and history
    pipeline_ = Pipeline([
        ('scaler', RobustScaler()),
        ('rf', RandomForestRegressor(**best_params_cleaned))
    ])

    # Save different model for each fold to make ensamble model later
    final_model.append(pipeline_.fit(X_train_1, y_train_1))

In [ ]:
y_pred_ensemble = np.zeros_like(y_test)

for model in final_model:
    y_pred = model.predict(X_test)
    y_pred_ensemble += y_pred

y_pred_ensemble /= len(final_model)

mee_ensemble = mean_euclidean_error(y_test, y_pred_ensemble)
r2_ensemble = multidim_r2(y_test, y_pred_ensemble)
rmse_ensemble = root_mean_squared_error(y_test, y_pred_ensemble)
mse_ensemble = mean_squared_error(y_test, y_pred_ensemble)

print('Ensemble MEE:', mee_ensemble)
print('Ensemble R2:', r2_ensemble)
print('Ensemble RMSE:', rmse_ensemble)
print('Ensemble MSE:', mse_ensemble)

# Ensemble of the final model using the whole dataset

In [ ]:
i = 0
final_model_ = []

for train_index, test_index in cv.split(X):
    X_train_1, X_test_1 = X[train_index], X[test_index]
    y_train_1, y_test_1 = y[train_index], y[test_index]
    i = i + 1

    # Generate 5 possible splits and foreach save a model and history
    pipeline_test = Pipeline([
        ('scaler', RobustScaler()),
        ('rf', RandomForestRegressor(**best_params_cleaned))
    ])

    # Save different model for each fold to make ensamble model later
    final_model_.append(pipeline_test.fit(X_train_1, y_train_1))

# Save model

In [ ]:
model_path = os.path.join(MODEL_FOLDER, 'random_forest_model.joblib')
dump(final_model, model_path, compress=3)

model_path = os.path.join(MODEL_FOLDER, 'random_forest_final_model.joblib')
dump(final_model_, model_path, compress=3)